# Chapter 16: Low-Rank Recovery from Linear Observations

In [1]:
import numpy as np
from numpy import linalg as LA
import cvxpy as cp
import time
from scipy.linalg import svd

In [2]:
# generate a rank-r matrix to be recovered from few linear observations
n = 60                                           # size of the square matrix
r = 3                                            # rank of the matrix
X = np.random.randn(n,r) @ np.random.rand(r,n)
m = 1000                                         # number of observations

## Recovery from generic observations via nuclear-norm minimization

Here, the observations made on the matrix $X\in\mathbb{R}^{n\times n}$ take the form
$$ y_i = \langle A_i,X\rangle_F = \mathrm{tr}(A_i^\top X), \quad i=1,\dots,m, $$
where $A_i,\dots,A_m\in\mathbb{R}^{n\times n}$ are gaussian matrices.

In [3]:
y_gen = np.zeros(m)
A = np.random.randn(m,n,n)
for i in range(m):
    y_gen[i] = np.trace(A[i].T@X)
t_gen_start = time.perf_counter()
M = cp.Variable((2*n,2*n),PSD=True)
objective = cp.Minimize(cp.trace(M))
constraints =  [cp.trace(A[i].T@ M[0:n,n:2*n]) == y_gen[i] for i in range(m)]
gen = cp.Problem(objective,constraints)
gen.solve(solver='SCS',eps=1e-6)
t_gen_stop = time.perf_counter()
t_gen = t_gen_stop - t_gen_start
X_gen = M.value[0:n,n:2*n]
rel_error_gen = LA.norm(X-X_gen,ord='fro')/LA.norm(X,ord='fro')
print('Recovery performed in {:.2f} sec with relative Frobenius-error of {:.2e} using nuclear-norm minimization'
      .format(t_gen,rel_error_gen))

Recovery performed in 48.29 sec with relative Frobenius-error of 8.39e-06 using nuclear-norm minimization


## Recovery from rank-one observations via nuclear-norm minimization

Here, the observations made on the matrix $X\in\mathbb{R}^{n\times n}$ take the form
$$ y_i = \langle b^{(i)},Xa^{(i)}\rangle = \mathrm{tr}\big( a^{(i)}b^{(i)^\top} X \big), \quad i=1,\dots,m,$$
where $a_1,\dots,a_m,b_1,\dots,b_m\in\mathbb{R}^n$ are gaussian vectors.

In [4]:
y_rk1 = np.zeros(m)
a = np.random.randn(n,m)
b = np.random.randn(n,m)
for i in range(m):
    y_rk1[i] = b[:,[i]].T@X@a[:,[i]]
t_rk1_start = time.perf_counter()
M = cp.Variable((2*n,2*n),PSD=True)
objective = cp.Minimize(cp.trace(M))
constraints = [b[:,[i]].T@M[0:n,n:2*n]@a[:,[i]] == y_rk1[i] for i in range(m)]
rk1 = cp.Problem(objective,constraints)
rk1.solve(solver='SCS',eps=1e-6)
t_rk1_stop = time.perf_counter()
t_rk1 = t_rk1_stop - t_rk1_start
X_rk1 = M.value[0:n,n:2*n]
rel_error_rk1 = LA.norm(X-X_rk1,ord='fro')/LA.norm(X,ord='fro')
print('Recovery performed in {:.2f} sec with relative Frobenius-error of {:.2e} using nuclear-norm minimization'
      .format(t_rk1, rel_error_rk1))

Recovery performed in 40.11 sec with relative Frobenius-error of 9.58e-07 using nuclear-norm minimization


## Recovery from rank-one observations via an iterative hard threholding algorithm

The above semidefinite-programming-based recovery procedure is prohibitively slow.
In case of rank-one observations, a faster strategy consists in iterating the following scheme:
$$ X_{k+1} = H_r\left(X_k + \mu_k\sum_{i=1}^{m}\mathrm{sgn}(y_i - \langle b^{(i)},X_ka^{(i)}\rangle)b^{(i)}a^{(i)^\top}\right), \quad \mu_k = \frac{ \sum_{i=1}^{m}|y_i - \langle b^{(i)},X_ka^{(i)}\rangle| }{\Big\| \sum_{i=1}^{m}\mathrm{sgn}(y_i - \langle b^{(i)},X_ka^{(i)}\rangle)b^{(i)}a^{(i)^\top} \Big\|_F^2} $$
The operator $H_r$ is the "hard thresholding operator" that returns the best rank-r approximant, i.e., the singular value decomposition truncated at level r. 
<br>
The theoretical justfication of this scheme's success can be found in [Foucart, Subramanian: Iterative hard thresholding for low-rank recovery from rank-one projections. Linear Algebra and its Applications, 572, 117--134, 2019].

In [5]:
nb_iter = 500
X_iht = np.zeros((n,n))
t_iht_start = time.perf_counter()
for k in range(nb_iter):
    residual = y_rk1 - np.sum( b * (X_iht @ a), axis=0 )       # this vector has entries y_i - < b^(i),X_k a^(i) >
    M = b @ np.diag( np.sign(residual) ) @ a.T                 # this matrix is \sum_i residual_i b^(i) a^(i)^T
    mu = LA.norm(residual,1) / LA.norm(M,ord='fro')**2
    U,D,V = svd(X_iht+mu*M)
    X_iht = U[:,:r] @ np.diag(D[:r]) @ V[:r,:]
t_iht_stop = time.perf_counter()
t_iht = t_iht_stop - t_iht_start
rel_error_iht = LA.norm(X-X_iht,ord='fro') / LA.norm(X,ord='fro')
print('Recovery performed in {:.2f} sec with relative Frobenius error of {:.2e} by iterative hard thresholding'
      .format(t_iht, rel_error_iht))

Recovery performed in 0.78 sec with relative Frobenius error of 6.25e-06 by iterative hard thresholding
